## Downloading All Packages Dependencies

In [6]:
import $ivy.`com.lihaoyi::requests:0.2.0`
import $ivy.`org.apache.kafka:kafka-clients:2.3.0`
import $ivy.`org.slf4j:slf4j-log4j12:1.7.25`

Downloaded https://repo1.maven.org/maven2/org/slf4j/slf4j-log4j12/1.7.25/slf4j-log4j12-1.7.25.pom
Downloaded https://repo1.maven.org/maven2/org/slf4j/slf4j-log4j12/1.7.25/slf4j-log4j12-1.7.25.pom.sha1
Downloaded https://repo1.maven.org/maven2/org/slf4j/slf4j-log4j12/1.7.25/slf4j-log4j12-1.7.25-sources.jar.sha1
Downloaded https://repo1.maven.org/maven2/org/slf4j/slf4j-log4j12/1.7.25/slf4j-log4j12-1.7.25-sources.jar
Downloaded https://repo1.maven.org/maven2/org/slf4j/slf4j-log4j12/1.7.25/slf4j-log4j12-1.7.25.jar.sha1
Downloaded https://repo1.maven.org/maven2/org/slf4j/slf4j-log4j12/1.7.25/slf4j-log4j12-1.7.25.jar


import $ivy.$                            

import $ivy.$                                     

import $ivy.$                               

## Importing all necessary modules

In [18]:
import org.slf4j.{Logger,LoggerFactory}
import org.apache.kafka.clients.producer.{KafkaProducer,ProducerConfig,ProducerRecord,Callback,RecordMetadata}

import java.sql.Date

import org.slf4j.{Logger,LoggerFactory}

import org.apache.kafka.clients.producer.{KafkaProducer,ProducerConfig,ProducerRecord,Callback,RecordMetadata}

In [41]:
import java.util.Properties

object StockProducer {
     
    val props = new Properties()
    props.put(ProducerConfig.VALUE_SERIALIZER_CLASS_CONFIG,"org.apache.kafka.common.serialization.StringSerializer")
    props.put(ProducerConfig.KEY_SERIALIZER_CLASS_CONFIG,"org.apache.kafka.common.serialization.StringSerializer")
    props.put(ProducerConfig.BOOTSTRAP_SERVERS_CONFIG,"localhost:8082")
    props.put(ProducerConfig.ACKS_CONFIG,"all")
    
    private val producer = new KafkaProducer[String,String](props)
    
    // Envia Mensagem utilizando a Key
    def sendMessage(kafkaTopic : String, key : String, message : String) : Unit = {
        
        var record : ProducerRecord[String,String] = new ProducerRecord[String,String](kafkaTopic,key,message)
        
        producer.send(record,new Callback {
            override def onCompletion(recordMetadata : RecordMetadata, e : Exception) : Unit = {
                if( e == null) {
                    println(e)
                } else {
                    println(s"Event Sent! : Partition : ${recordMetadata.offset()}, Enqeued Time : ${recordMetadata.timestamp}")
                }
            }
        })
    }
    
    // Envia Mensagem utilizando Round Robin Strategy
    def sendMessage(kafkaTopic : String, message : String) : Unit = {
        
        var record : ProducerRecord[String,String] = new ProducerRecord[String,String](kafkaTopic,message)
        
        producer.send(record,new Callback {
            override def onCompletion(recordMetadata : RecordMetadata, e : Exception) : Unit = {
                if( e == null) {
                    println(e)
                } else {
                    println(s"Event Sent! : Partition : ${recordMetadata.offset()}, Enqeued Time : ${recordMetadata.timestamp}")
                }
            }
        })
    }
    
}

object Requester {

    private final val apiKey : String = "CW7DIEFXLWKPC797"
    private val url : String = "https://www.alphavantage.co/query?function=GLOBAL_QUOTE"
    
    def get(stockName : String, interval : Long = 120000) : Unit = {
        val finalUrl : String = this.url + s"&symbol=${stockName}" + s"&apikey=${this.apiKey}"
        
        while(true) {
            val message = requests.get(finalUrl).text
//             StockProducer.sendMessage("stock_topic",stockName,message)
            println(message)
            Thread.sleep(interval)
        }
        
    }
}

import java.util.Properties


defined object StockProducer
defined object Requester

In [42]:
class StockThread extends Runnable {
    
    var stockName : String = _
    var interval : Long = _
    
    def this(stockName : String, interval : Long = 60000) {
        this()
        this.stockName = stockName
        this.interval = interval
    }
    
    def run() : Unit = {
        Requester.get(this.stockName,this.interval)
    }
    
}

defined class StockThread

In [43]:
val thread1 = new Thread(new StockThread("MSFT"))
val thread2 = new Thread(new StockThread("FB"))
val thread3 = new Thread(new StockThread("AAPL"))

thread1: Thread = Thread[Thread-3,5,main]
thread2: Thread = Thread[Thread-4,5,main]
thread3: Thread = Thread[Thread-5,5,main]

In [44]:
thread1.start

In [45]:
thread2.start

In [46]:
thread3.start